In [1]:
import requests
from requests.api import request
import time
import json
import re
import random
import os
import pickle
import trio
import subprocess
import shutil
from enum import Enum
# import win32file
# win32file._setmaxstdio(8192)

PREVIOUS_REQUESTS_BACKUP_PATH = "./previousRequests/"


def get_valid_filename(name):
    s = str(name).strip().replace(" ", "_")
    s = re.sub(r"(?u)[^-\w.]", "", s)
    if s in {"", ".", ".."}:
        return "ErrorFileName" + str(random.randrange(100000))
    return s


def get_valid_path(path):
    s = str(path).strip().replace(" ", "_")
    s = re.sub(r"(?u)[^-\w./]", "", s)
    if s in {"", ".", ".."}:
        return "ErrorFileName" + str(random.randrange(100000))
    if s[-1] == ".":
        s += "_"
    return s


def dumpAPIRequest(fileName, data, path=PREVIOUS_REQUESTS_BACKUP_PATH):
    fullFileName = get_valid_path(path) + get_valid_filename(fileName) + ".bin"
    os.makedirs(os.path.dirname(fullFileName), exist_ok=True)
    with open(fullFileName, "wb") as outputFile:
        pickle.dump(data, outputFile)
    return fullFileName


def loadAPIRequest(fileName, path=PREVIOUS_REQUESTS_BACKUP_PATH):
    fullFileName = path + get_valid_filename(fileName) + ".bin"
    if not os.path.exists(fullFileName):
        return {}
    with open(fullFileName, "rb") as inputFile:
        data = pickle.load(inputFile)
    return data


def checkForAPIRequest(url, path=PREVIOUS_REQUESTS_BACKUP_PATH):
    return os.path.exists(
        path + get_valid_filename(url) + ".bin"
    )


def dumpJSON(fileName, data, path="./"):
    fullFileName = get_valid_path(path) + get_valid_filename(fileName) + ".json"
    os.makedirs(os.path.dirname(fullFileName), exist_ok=True)
    while len(fileName) > 1:
        try:
            with open(fullFileName, "w") as outputFile:
                json.dump(data, outputFile, indent=6)
            return fullFileName
        except:
            fileName = fileName[:-1]
            fullFileName = get_valid_path(path) + get_valid_filename(fileName) + ".json"
    


def loadJSON(fullFileName):
    if not os.path.exists(fullFileName):
        return {}
    with open(fullFileName, "r") as inputFile:
        data = json.load(inputFile)
    return data


def checkForJSON(fileName, path):
    fullFileName = get_valid_path(path) + get_valid_filename(fileName) + ".json"
    return loadJSON(fullFileName) if os.path.exists(fullFileName) else {}
    # return fullFileName if os.path.exists(fullFileName) else {}


def dumpGeneric(fileName, list, type="txt", path="./"):
    fullFileName = get_valid_path(path) + get_valid_filename(fileName) + "." + type
    os.makedirs(os.path.dirname(fullFileName), exist_ok=True)
    with open(fullFileName, "w") as outputFile:
        for item in list:
            outputFile.write(str(item) + "\n")
    return fullFileName


def nurseryReturn(nursery, assignee, location, func, *args):
    async def getReturn():
        assignee[location] = await func(*args)

    nursery.start_soon(getReturn)

In [2]:
SCRATCH_API = "https://api.scratch.mit.edu"
WAIT_TIME = 0.1
lastRequestTime = 0
timerLock = trio.Lock()
markLock = trio.Lock()
markedRequests = set({})
imageLock = trio.Lock()
markedImages = set({})
AUTH_ADDITION = "?x-token="
requestLimit = trio.CapacityLimiter(100)
progressChecker = 0



async def apiRequest(url):
    if checkForAPIRequest(url):
        return loadAPIRequest(url)
    
    async with markLock:
        marked = url in markedRequests
        if not marked:
            markedRequests.add(url)
    if marked:
        while url in markedRequests:
            await trio.sleep(1)
        if checkForAPIRequest(url):
            return loadAPIRequest(url)
    
    async with requestLimit:
        async with timerLock:
            global lastRequestTime
            sleepTime = max(lastRequestTime + WAIT_TIME - time.time(), 0)
            await trio.sleep(sleepTime)
            lastRequestTime = time.time()

        response = requests.get(url)
        data = response.json() if response.ok else {}

        dumpAPIRequest(url, data)
        async with markLock:
            markedRequests.remove(url)

        global progressChecker
        progressChecker += 1
        if progressChecker % 100 == 0:
            print(progressChecker)

        return data

async def imageGet(url, path="./", backupPath = PREVIOUS_REQUESTS_BACKUP_PATH):
    async with imageLock:
        marked = url in markedImages
        if not marked:
            markedImages.add(url)
    if marked:
        while url in markedImages:
            await trio.sleep(0.1)
    
    imageName = re.search("[^/]*.png", url).group()
    imageBackupPath = backupPath + imageName

    if not os.path.exists(imageBackupPath):
        imageRequest = requests.get(url, stream=True)
        if imageRequest.status_code == 200:
            os.makedirs(backupPath, exist_ok=True)
            with open(imageBackupPath, 'wb') as imageFile:
                imageRequest.raw.decode_content = True
                shutil.copyfileobj(imageRequest.raw, imageFile)
            markedImages.remove(url)
        else:
            markedImages.remove(url)
            print("Image not found: " + url)
            return

    imagePath = path + imageName
    os.makedirs(path, exist_ok=True)
    shutil.copy(imageBackupPath, imagePath)


async def getAllResults(url):
    limit = 40
    url = url + "?limit=" + str(limit) + "&offset="
    offset = 0
    singleList = await apiRequest(url + str(offset))
    all = singleList
    while len(singleList) >= limit:
        offset += limit
        singleList = await apiRequest(url + str(offset))
        if len(singleList) > 0:
            all += singleList
    return all


async def getAllResultsDateBased(url):
    limit = 40
    url = url + "?limit=" + str(limit)
    singleList = await apiRequest(url)
    all = singleList
    while len(singleList) >= limit:
        dateLimit = all[-1].datetime_created
        singleList = await apiRequest(url + "?dateLimit=" + str(dateLimit))
        if len(singleList) > 0:
            overlapIndex = singleList.index(all[-1]) + 1
            if overlapIndex < len(singleList):
                truncatedList = singleList[(singleList.index(all[-1]) + 1) :]
                if len(truncatedList) > 0:
                    all += truncatedList
    return all


async def getCommentsWithReplies(url):
    comments = await getAllResults(url)
    async with trio.open_nursery() as nursery:
        for comment in comments:
            if "reply_count" in comment.keys() and comment["reply_count"] > 0:
                # async def getReplies():
                #     comment["replies"] = await getAllResults(url + "/" + str(comment["id"]) + "/replies")
                # nursery.start_soon(getReplies)
                nurseryReturn(
                    nursery,
                    comment,
                    "replies",
                    getAllResults,
                    url + "/" + str(comment["id"]) + "/replies",
                )

            else:
                comment["replies"] = {}
                comment["reply_count"] = 0
    return comments

In [3]:
# project calls

projectsInfoAPIAddition = "/projects/"
projectInfoAPI = SCRATCH_API + projectsInfoAPIAddition

async def getProjectInfo(projectID):
    return await apiRequest(projectInfoAPI + str(projectID))

projectRemixesAPIAddition = "/remixes"
async def getProjectRemixes(projectID):
    return await getAllResults(projectInfoAPI + str(projectID) + projectRemixesAPIAddition)


# bonus helper function
userIDs = {}
async def getProjectUserName(projectID, userID=None):
    if userID is None or userID not in userIDs.keys():
        projectInfo = await getProjectInfo(projectID)
        if(projectInfo != {}):
            userIDs[userID] = projectInfo["author"]["username"]
            return userIDs[userID]
        else:
            return ""
    else:
        return userIDs[userID]


In [4]:
# studio calls

studioInfoAPIAddition = "/studios/"
studioInfoAPI = SCRATCH_API + studioInfoAPIAddition
async def getStudioInfo(studioID):
    return await apiRequest(studioInfoAPI + str(studioID))

studioActivityAPIAddition = "/activity"
async def getStudioActivity(studioID):
    return await getAllResults(studioInfoAPI + str(studioID) + studioActivityAPIAddition)

studioCommentsAPIAddition = "/comments"
async def getStudioComments(studioID):
    return await getCommentsWithReplies(studioInfoAPI + str(studioID) + studioCommentsAPIAddition)

studioCuratorsAPIAddition = "/curators"
async def getStudioCurators(studioID):
    return await getAllResults(studioInfoAPI + str(studioID) + studioCuratorsAPIAddition)

studioManagersAPIAddition = "/managers"
async def getStudioManagers(studioID):
    return await getAllResults(studioInfoAPI + str(studioID) + studioManagersAPIAddition)

studioProjectsAPIAddition = "/projects"
async def getStudioProjects(studioID):
    studioProjects = await getAllResults(studioInfoAPI + str(studioID) + studioProjectsAPIAddition)
    # async with trio.open_nursery() as nursery:
    #     for i in range(len(studioProjects)):
    #         nurseryReturn(nursery, studioProjects, i, getProjectInfo, studioProjects[i]["id"])
    return studioProjects

studioUserRoleAPIAddition = "/users/"
async def getStudioUserRole(studioID, userName, authToken):
    return await getAllResults(studioInfoAPI + str(studioID) + studioUserRoleAPIAddition + userName + AUTH_ADDITION + authToken)

In [5]:
# user calls

userInfoAPIAddition = "/users/"
userInfoAPI = SCRATCH_API + userInfoAPIAddition
async def getUserInfo(userName):
    return await apiRequest(userInfoAPI + str(userName))

userFavoritesAPIAddition = "/favorites"
async def getUserFavorites(userName):
    projects = await getAllResults(userInfoAPI + str(userName) + userFavoritesAPIAddition)
    async with trio.open_nursery() as nursery:
        for project in projects:
            nurseryReturn(nursery, project["author"], "username", getProjectUserName, project["id"], project["author"]["id"])
    return projects

userFollowersAPIAddition = "/followers"
async def getUserFollowers(userName):
    return await getAllResults(userInfoAPI + str(userName) + userFollowersAPIAddition)

userFollowingAPIAddition = "/following"
async def getUserFollowing(userName):
    return await getAllResults(userInfoAPI + str(userName) + userFollowingAPIAddition)

userFollowingStudiosAPIAddition = userFollowingAPIAddition + "/studios/projects"
async def getUserFollowingStudios(userName, authToken):
    return await getAllResults(userInfoAPI + str(userName) + userFollowingStudiosAPIAddition + AUTH_ADDITION + authToken)

userFollowingUsersActivityAPIAddition = userFollowingAPIAddition + "/users/activity"
async def getUserFollowingUsersActivity(userName, authToken):
    return await getAllResults(userInfoAPI + str(userName) + userFollowingUsersActivityAPIAddition + AUTH_ADDITION + authToken)

userFollowingUsersLovesAPIAddition = userFollowingAPIAddition + "/users/loves"
async def getUserFollowingUsersLoves(userName, authToken):
    return await getAllResults(userInfoAPI + str(userName) + userFollowingUsersLovesAPIAddition + AUTH_ADDITION + authToken)

userFollowingUsersProjectsAPIAddition = userFollowingAPIAddition + "/users/projects"
async def getUserFollowingUsersProjects(userName, authToken):
    return await getAllResults(userInfoAPI + str(userName) + userFollowingUsersProjectsAPIAddition + AUTH_ADDITION + authToken)

userInvitesAPIAddition = "/invites"
async def getUserInvites(userName, authToken):
    return await getAllResults(userInfoAPI + str(userName) + userInvitesAPIAddition + AUTH_ADDITION + authToken)

userMessagesAPIAddition = "/messages"
async def getUserMessages(userName, authToken):
    return await getAllResults(userInfoAPI + str(userName) + userMessagesAPIAddition + AUTH_ADDITION + authToken)

userAlertsAPIAddition = "/messages/admin"
async def getUserAlerts(userName, authToken):
    return await getAllResults(userInfoAPI + str(userName) + userAlertsAPIAddition + AUTH_ADDITION + authToken)

userUnreadMessagesCountAPIAddition = "/messages/count"
async def getUserUnreadMessagesCount(userName, authToken):
    return await getAllResults(userInfoAPI + str(userName) + userUnreadMessagesCountAPIAddition + AUTH_ADDITION + authToken)

userProjectsAPIAddition = "/projects"
async def getUserProjects(userName):
    projects = await getAllResults(userInfoAPI + str(userName) + userProjectsAPIAddition)
    for project in projects:
        project["author"]["username"] = userName
    return projects

userRecentlyViewedAPIAddition = "/projects/recentlyviewed"
async def getUserRecentlyViewed(userName, authToken):
    return await getAllResults(userInfoAPI + str(userName) + userRecentlyViewedAPIAddition + AUTH_ADDITION + authToken)

userProjectCommentsAPIAddition = "/comments"
userProjectsAPIAdditionMiddle = "/projects/"
async def getUserProjectComments(userName, projectID):
    return await getCommentsWithReplies(userInfoAPI + str(userName) + userProjectsAPIAdditionMiddle + str(projectID) + userProjectCommentsAPIAddition)

userProjectStudiosAPIAddition = "/studios"
async def getUserProjectStudiosAPI(userName, projectID):
    return await getAllResults(userInfoAPI + str(userName) + userProjectsAPIAdditionMiddle + str(projectID) + userProjectStudiosAPIAddition)

userUnsharedProjectAPIAddition = "/visibility"
async def getUserUnsharedProject(userName, projectID):
    return await getCommentsWithReplies(userInfoAPI + str(userName) + userProjectsAPIAdditionMiddle + str(projectID) + userUnsharedProjectAPIAddition)

userStudiosAPIAddition = "/studios/curate"
async def getUserStudios(userName):
    return await getAllResults(userInfoAPI + str(userName) + userStudiosAPIAddition)

In [6]:
projectsToDownload = {}
urlsToDownload = set({})

scratchBaseURL = "https://scratch.mit.edu/"
userURL = scratchBaseURL + "users/"
projectURL = scratchBaseURL + "projects/"
studioURL = scratchBaseURL + "studios/"

levelCountLock = trio.Lock()
levelCount = {}

class LevelCountTypes(Enum):
    DONE = "Done"
    TOTAL = "Total"

async def handleLevelCount(level, countType, outputPath=""):
    async with levelCountLock:
        if level not in levelCount.keys():
            levelCount[level] = {}
            for levelCountType in LevelCountTypes:
                levelCount[level][levelCountType] = {"Count": 0, "Lock": trio.Lock()}
    async with levelCount[level][countType]["Lock"]:
        levelCount[level][countType]["Count"] += 1
    print(("\t"*level) + str(levelCount[level][LevelCountTypes.DONE]["Count"]) + " / " + str(levelCount[level][LevelCountTypes.TOTAL]["Count"]) + "\t" + outputPath)


async def getProjectData(
    userName, projectID, level=0, outputPath="./", authUserName=None, authToken=None
):
    urlsToDownload.add(projectURL + str(projectID))

    project = {}
    project["projectInfo"] = await getProjectInfo(projectID)
    if project["projectInfo"] == {}:
        return ""


    outputPath += get_valid_path(project["projectInfo"]["title"]) + "/"
    print(outputPath)

    await imageGet(project["projectInfo"]["image"], outputPath)

    if projectID not in projectsToDownload.keys():
        projectsToDownload[projectID] = []

    projectsToDownload[projectID].append(outputPath)


    fileName = get_valid_filename(project["projectInfo"]["title"])

    jsonCheck = checkForJSON(fileName, outputPath)
    if jsonCheck != {} and jsonCheck["level"] >= level:
        return jsonCheck

    await handleLevelCount(level, LevelCountTypes.TOTAL)

    async with trio.open_nursery() as nursery:
        nurseryReturn(nursery, project, "projectRemixes", getProjectRemixes, projectID)
        nurseryReturn(
            nursery,
            project,
            "projectStudios",
            getUserProjectStudiosAPI,
            userName,
            projectID,
        )
        nurseryReturn(
            nursery,
            project,
            "projectComments",
            getUserProjectComments,
            userName,
            projectID,
        )


    if level > 0:
        newLevel = level - 1

        async with trio.open_nursery() as nursery:
            for projectRemix in project["projectRemixes"]:
                nurseryReturn(
                    nursery,
                    projectRemix,
                    "projectData",
                    getProjectData,
                    projectRemix["author"]["username"],
                    projectRemix["id"],
                    newLevel,
                    outputPath + "remixes/",
                    authUserName,
                    authToken,
                )
            for projectStudio in project["projectStudios"]:
                nurseryReturn(
                    nursery,
                    projectStudio,
                    "studioData",
                    getStudioData,
                    projectStudio["id"],
                    newLevel,
                    outputPath + "studios/",
                    authUserName,
                    authToken,
                )
            for projectComment in project["projectComments"]:
                nurseryReturn(
                    nursery,
                    projectComment,
                    "userData",
                    getUserData,
                    projectComment["author"]["username"],
                    newLevel,
                    outputPath + "commentUsers/",
                    authUserName,
                    authToken,
                )

    project["level"] = level
    outputFileName = dumpJSON(fileName, project, outputPath)
    await handleLevelCount(level, LevelCountTypes.DONE, outputPath)
    return outputFileName



async def getStudioData(
    studioID, level=0, outputPath="./", authUserName=None, authToken=None
):
    urlsToDownload.add(studioURL + str(studioID))

    studio = {}
    studio["studioInfo"] = await getStudioInfo(studioID)


    outputPath += get_valid_path(studio["studioInfo"]["title"]) + "/"
    print(outputPath)

    await imageGet(studio["studioInfo"]["image"], outputPath)

    fileName = get_valid_filename(studio["studioInfo"]["title"])

    jsonCheck = checkForJSON(fileName, outputPath)
    if jsonCheck != {} and jsonCheck["level"] >= level:
        return jsonCheck
    
    await handleLevelCount(level, LevelCountTypes.TOTAL)


    async with trio.open_nursery() as nursery:
        nurseryReturn(nursery, studio, "studioActivity", getStudioActivity, studioID)
        nurseryReturn(nursery, studio, "studioComments", getStudioComments, studioID)
        nurseryReturn(nursery, studio, "studioCurators", getStudioCurators, studioID)
        nurseryReturn(nursery, studio, "studioManagers", getStudioManagers, studioID)
        nurseryReturn(nursery, studio, "studioProjects", getStudioProjects, studioID)
        if authUserName is not None:
            nurseryReturn(
                nursery,
                studio,
                "studioUserRole",
                getStudioUserRole,
                studioID,
                authUserName,
                authToken,
            )


    if level > 0:
        newLevel = level - 1

        async with trio.open_nursery() as nursery:
            for studioCurator in studio["studioCurators"]:
                nurseryReturn(
                    nursery,
                    studioCurator,
                    "userData",
                    getUserData,
                    studioCurator["username"],
                    newLevel,
                    outputPath + "curators/",
                    authUserName,
                    authToken,
                )
            for studioManager in studio["studioManagers"]:
                nurseryReturn(
                    nursery,
                    studioManager,
                    "userData",
                    getUserData,
                    studioManager["username"],
                    newLevel,
                    outputPath + "managers/",
                    authUserName,
                    authToken,
                )
            for studioProject in studio["studioProjects"]:
                nurseryReturn(
                    nursery,
                    studioProject,
                    "projectData",
                    getProjectData,
                    studioProject["username"], # studioProject["author"]["username"],
                    studioProject["id"],
                    newLevel,
                    outputPath + "projects/",
                    authUserName,
                    authToken,
                )

    studio["level"] = level
    outputFileName = dumpJSON(fileName, studio, outputPath)
    await handleLevelCount(level, LevelCountTypes.DONE, outputPath)
    return outputFileName



async def getUserData(
    userName, level=0, outputPath="./", authUserName=None, authToken=None,
):
    urlsToDownload.add(userURL + userName)

    user = {}
    user["userInfo"] = await getUserInfo(userName)


    outputPath += get_valid_path(userName) + "/"
    print(outputPath)

    if("profile" not in user["userInfo"]):
        print(userName)
        print(user["userInfo"])
    await imageGet(user["userInfo"]["profile"]["images"]["90x90"], outputPath)

    fileName = get_valid_filename(userName)

    jsonCheck = checkForJSON(fileName, outputPath)
    if jsonCheck != {} and jsonCheck["level"] >= level:
        return jsonCheck
    
    await handleLevelCount(level, LevelCountTypes.TOTAL)

    async with trio.open_nursery() as nursery:
        nurseryReturn(nursery, user, "userFavorites", getUserFavorites, userName)
        nurseryReturn(nursery, user, "userFollowers", getUserFollowers, userName)
        nurseryReturn(nursery, user, "userFollowing", getUserFollowing, userName)
        nurseryReturn(nursery, user, "userProjects", getUserProjects, userName)
        nurseryReturn(nursery, user, "userStudios", getUserStudios, userName)
        if userName == authUserName:
            frontPageStuffName = "userCurrentFrontPage"
            user[frontPageStuffName] = {}
            nurseryReturn(
                nursery,
                user[frontPageStuffName],
                "Projects in Studios I'm Following",
                getUserFollowingStudios,
                authUserName,
                authToken,
            )
            nurseryReturn(
                nursery,
                user[frontPageStuffName],
                "What's Happening?",
                getUserFollowingUsersActivity,
                authUserName,
                authToken,
            )
            nurseryReturn(
                nursery,
                user[frontPageStuffName],
                "Projects Loved by Scratchers I'm Following",
                getUserFollowingUsersLoves,
                authUserName,
                authToken,
            )
            nurseryReturn(
                nursery,
                user[frontPageStuffName],
                "Projects by Scratchers I'm Following",
                getUserFollowingUsersProjects,
                authUserName,
                authToken,
            )
            notifications = "userNotifications"
            user[notifications] = {}
            nurseryReturn(
                nursery,
                user[notifications],
                "invites",
                getUserInvites,
                authUserName,
                authToken,
            )
            nurseryReturn(
                nursery,
                user[notifications],
                "messages",
                getUserMessages,
                authUserName,
                authToken,
            )
            nurseryReturn(
                nursery,
                user[notifications],
                "alerts",
                getUserAlerts,
                authUserName,
                authToken,
            )
            nurseryReturn(
                nursery,
                user[notifications],
                "unreadMessagesCount",
                getUserUnreadMessagesCount,
                authUserName,
                authToken,
            )
            nurseryReturn(
                nursery,
                user,
                "userRecentlyViewed",
                getUserRecentlyViewed,
                authUserName,
                authToken,
            )
    
    if level > 0:
        newLevel = level - 1

        async with trio.open_nursery() as nursery:
            for userFavorite in user["userFavorites"]:
                nurseryReturn(
                    nursery,
                    userFavorite,
                    "projectData",
                    getProjectData,
                    userFavorite["author"]["username"],
                    userFavorite["id"],
                    newLevel,
                    outputPath + "favorites/",
                    authUserName,
                    authToken,
                )
            for userFollower in user["userFollowers"]:
                nurseryReturn(
                    nursery,
                    userFollower,
                    "userData",
                    getUserData,
                    userFollower["username"],
                    newLevel,
                    outputPath + "followers/",
                    authUserName,
                    authToken,
                )
            for userFollow in user["userFollowing"]:
                nurseryReturn(
                    nursery,
                    userFollow,
                    "userData",
                    getUserData,
                    userFollow["username"],
                    newLevel,
                    outputPath + "following/",
                    authUserName,
                    authToken,
                )
            for userProject in user["userProjects"]:
                nurseryReturn(
                    nursery,
                    userProject,
                    "projectData",
                    getProjectData,
                    userProject["author"]["username"],
                    userProject["id"],
                    newLevel,
                    outputPath + "projects/",
                    authUserName,
                    authToken,
                )
            for userStudio in user["userStudios"]:
                nurseryReturn(
                    nursery,
                    userStudio,
                    "studioData",
                    getStudioData,
                    userStudio["id"],
                    newLevel,
                    outputPath + "studios/",
                    authUserName,
                    authToken,
                )

    user["level"] = level
    outputFileName = dumpJSON(fileName, user, outputPath)
    await handleLevelCount(level, LevelCountTypes.DONE, outputPath)
    return outputFileName

In [7]:
# MARK_DATA = {"mark"}

# for fileName in os.listdir(PREVIOUS_REQUESTS_BACKUP_PATH):
#     fileDirectory = os.path.join(PREVIOUS_REQUESTS_BACKUP_PATH, fileName)
#     delete = False
#     with open(fileDirectory, "rb") as inputFile:
#         delete = pickle.load(inputFile) == MARK_DATA
#     if delete:
#         os.remove(fileDirectory)

In [8]:
projectsToDownload = {}
urlsToDownload = set({})

In [9]:
authData = loadJSON("scratchAuthenticationToken.json")
if(authData["x-token"] == None):
    trio.run(getUserData, authData["username"], authData["level"], "./")
else:
    trio.run(getUserData, authData["username"], authData["level"], "./", authData["username"], authData["x-token"])

./swifty2/
	0 / 1	
./swifty2/favorites/Go_Away--Coloring_Contest_remix/
./swifty2/favorites/BUTTONS_FOR_EVERYONE/
./swifty2/favorites/I_cant_believe_its_not_art/
./swifty2/favorites/Fanart_for_swifty_and_sonicqueen/
./swifty2/favorites/Fanart_for_Swifty2/
./swifty2/favorites/Swifty2_Fanart/
./swifty2/favorites/Swifty2_fanart_/
./swifty2/favorites/Art_trades__stuff/
./swifty2/favorites/Fanart_for_Swiftay/
./swifty2/favorites/unfinishedThe_Target-Da_Anime_Way/
./swifty2/favorites/Scratchers_To_Mobianspart_1/
./swifty2/favorites/MyLife_Episode_10_Part_1/
./swifty2/favorites/SWIFTY_DUBSTEP/
./swifty2/favorites/WTF/
./swifty2/favorites/swifty/
./swifty2/favorites/My_Entry_To_Swiftys_Contest/
./swifty2/favorites/Swiftys_Contest_entry/
./swifty2/favorites/Contest_Entry_for_Swifty2/
./swifty2/favorites/entry_for_swifty2/
./swifty2/favorites/My_Entry_in_Swiftys_Icon_Contest/
./swifty2/favorites/Swifty2_EPIC_Contest_Entry_thmsrox590/
./swifty2/favorites/Swifty_Contest_Entry__This_is_for_you_swif

In [ ]:
dumpJSON(authData["username"] + "_projects", projectsToDownload)
dumpGeneric(authData["username"] + "_urls", urlsToDownload, type="txt")

'./swifty2_urls.txt'

In [ ]:
p = subprocess.Popen(['node', './downloadProject.js'], stdout=subprocess.PIPE)
for line in iter(p.stdout.readline, b""):
        print(line.decode(), end="")

Done
